In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import json
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statistics import mode

# Thanks StackOverflow!
# https://stackoverflow.com/questions/21137150/format-suppress-scientific-
#         notation-from-python-pandas-aggregation-results
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
cd '/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project/analysis'

In [ ]:
conn = sqlite3.connect('movie_analysis2.db')
cur = conn.cursor()

In [ ]:
# Source: https://pbpython.com/effective-matplotlib.html

# {:1.1f} in this case means: apply format to the first 1 instance
# that value should be presented with .1f = as a float with 1 point precision
# change the second number to increase or decrease precision

def currency(x, pos=None):
    'The two args are the value and tick position'
    # over 1 billion
    if abs(x) >= 1000000000:
        return '${:1.2f} B'.format(x*1e-9)
    # over 10 million
    elif abs(x) >= 10000000:
        return '${:1.1f} M'.format(x*1e-6)
    # over 1 million
    elif abs(x) >= 1000000:
        return '${:1.2f} M'.format(x*1e-6)
    elif x == 0:
        return '${:0}'.format(x)
    else:
        return '${:1.0f} K'.format(x*1e-3)
    

# Topic 1: Profitability Trends

## Create dataframe with relevant films for financial analysis

In [ ]:
# Updated query with already-filtered movies

sql = """SELECT DISTINCT m.id
                        , m.budget
                        , m.revenue
                        , m.release_date
                        , m.original_language
                        , m.imdb_id
                        , m.runtime
                        , m.vote_average
                        , m.vote_count
         FROM tmdb_movies m
         """

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])

In [ ]:
# check column types and counts
df.info()

In [ ]:
# check the min, max values for numerical columns
df.describe()

We definitely have some movies where budget or revenue is missing, or one or the other are really low and probably not accurate.
Let's drop rows from the table where the budget and revenue data doesn't look usable.

Will use a cut off limit of \\$10,000; both budget and revenue must be over \\$10,000


- https://www.themoviedb.org/movie/599290-wwr-tournament-for-tomorrow is a good example
- https://www.themoviedb.org/movie/254191-honeymoon another example

In [ ]:
# Export to CSV to review before dropping. Everything here looks wonky.
df.loc[(df['budget'] < 10000) | (df['revenue'] < 10000),['budget','revenue']
      ].to_csv('budget_revenue_double-check.csv')

In [ ]:
# Drop the rows where either budget or revennue is less than 10k
to_drop = df[(df['budget'] < 10000) | (df['revenue'] < 10000)].index
df.drop(labels=to_drop, axis=0, inplace=True)


In [ ]:
# 4708 records dropped, and 1787 remain
df.info()

In [ ]:
# min budget and revenues look better
df.describe()

In [ ]:
# But runtime has a min of 0. Let's replace with NaN so they won't be calculated
df.loc[df['runtime'] == 0, 'runtime'] = np.nan

In [ ]:
# Now there should be no 0 min for runtime
df.describe()

In [ ]:
# Looks like all of our films have release dates (string len 10)
# This makes sense, since we filtered the original set of movie ids
# based on release dates 2010 and on
df['release_date'].map(lambda x: len(x)).value_counts()

In [ ]:
# Split out year and month into new df columns
month_map = {1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',
             7:'July',8:'August',9:'September',10:'October',11:'November',
             12:'December'}

df['release_year'] = df[df['release_date'].isna()==False]['release_date'].map(lambda x: int(x[:4]))
df['release_month'] = df[df['release_date'].isna()==False]['release_date'].map(lambda x: month_map[int(x[5:7])])

# Would need to use the below if there were null release dates, but there aren't
#df.loc[df['release_year'].isna()==True, 'release_year'] = 0
#df['release_year'] = df['release_year'].astype(int)

In [ ]:
# add a numerical month column, to make sorting easier
rev_month_map = {}
for k, v in month_map.items():
    rev_month_map[v] = k
    
rev_month_map

df['release_month_num'] = df['release_month'].map(lambda x: rev_month_map[x])


In [ ]:
df.head()

Looks like I have movies that were released this year, in 2021. These results are incomplete since we're still in 2021, so I will drop those years from this dataset for now. 

In [ ]:
# Drop the rows where either budget or revennue is less than 10k
to_drop = df[df['release_year'] == 2021].index
df.drop(labels=to_drop, axis=0, inplace=True)

In [ ]:
# Check to see they were all dropped out, this should return nothing
df[df['release_year'] == 2021]

Before calculating profitability, want to adjust the numbers for inflation. Retrieved the Consumer Price Index from http://www.bls.gov/ using the instructions here: https://www.maa.org/press/periodicals/loci/joma/the-consumer-price-index-and-inflation-get-cpi-data-from-the-web-and-into-a-spreadsheet

Will make the assumption that the vast majority of the revenue happened in the release year. This assumption is based on the revenue being box office only, and typically most movies only play in theaters for a few months. The one caveat is that revenue is worlwide, and releases in other countries may have been somewhat later.

In [ ]:
cd '/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project/data'

In [ ]:
# Read in Consumer Price Index info from CSV
df_cpi = pd.read_csv('Consumer Price Index.csv')

cpi_dict = pd.Series(df_cpi['Annual'].values,index=df_cpi['Year']).to_dict()

In [ ]:
# Adjust budget for inflation
df['budget_2020adj'] = df[(df['budget'].isna()==False) & (df['budget'] > 0)
       ][['budget', 'release_year']].apply(
        lambda x : (x[0] * cpi_dict[2020]) / cpi_dict[x[1]], axis=1)


In [ ]:
# Adjust revenue for inflation
df['revenue_2020adj'] = df[(df['revenue'].isna()==False) & (df['revenue'] > 0)
       ][['revenue', 'release_year']].apply(
        lambda x : (x[0] * cpi_dict[2020]) / cpi_dict[x[1]]
                                           , axis=1)

In [ ]:
# Calculate profit with inflation-adjusted values
# Only calculate where have both sets of values
df['profit_2020adj'] = df[(df['revenue_2020adj'].isna()==False) 
                      & (df['budget_2020adj'].isna()==False)
       ][['revenue_2020adj', 'budget_2020adj']].apply(lambda x : (x[0] - x[1]), axis=1)

In [ ]:
# Calculate % profit with inflation-adjusted values
df['profitpercent_2020adj'] = df[(df['revenue_2020adj'].isna()==False) 
                      & (df['budget_2020adj'].isna()==False)
       ][['revenue_2020adj', 'budget_2020adj']].apply(
    lambda x : ((x[0] - x[1]) / x[1]) * 100, axis=1)

In [ ]:
# Hmm, so we have instances of profit being negative almost 100%, but not 
# quite -100%. This is important, since a profit of -100% would mean made $0
# or very little in revenue, and we were supposed to have removed the small values.

# Mean profit percentage is 327%
df['profitpercent_2020adj'].describe()

In [ ]:
# And median profit percentage is 121%, which means doubled money + 20%
df['profitpercent_2020adj'].median()

In [ ]:
df[df['profitpercent_2020adj'] < 0]['profitpercent_2020adj'].sort_values()

Looks like we have a pretty good size sample of movies for each year. 2019 is smaller and 2020 even smaller than that. It could be there were truly fewer movies released to theaters in 2020 due to COVID-19, and could also be that it takes a while for users to enter revenues and budgets, and that 

In [ ]:
# How many movies do we have in our sample for each year?
df['release_year'].value_counts(sort=False)

In [ ]:
# OK so we don't have any outliers on the lower end, but we definitely do on the upper end.
# I'm going to separate into
# less than 0 (lost money)
# 0-200% profit
# 200-400% profit
# 400-600% profit
# 600-817% profit
# everything above 817%, which are outliers

q1 = df['profitpercent_2020adj'].quantile(.25)
q3 = df['profitpercent_2020adj'].quantile(.75)
iqr = q3-q1

lower_outlier_bound = q1-(iqr*1.5)
upper_outlier_bound = q3+(iqr*1.5)

display(lower_outlier_bound, upper_outlier_bound)

In [ ]:
q10_labels = ['Percentile 0-10','Percentile 10-20','Percentile 20-30', 
              'Percentile 30-40', 'Percentile 40-50', 'Percentile 50-60', 
              'Percentile 60-70', 'Percentile 70-80', 'Percentile 80-90', 
              'Percentile 90-100']

q4_labels = ['Bottom 25%', '25 to 50 Percentiles', '50 to 75 Percentiles', 'Top 25%']

In [ ]:
df['roi_quantile'] = pd.qcut(df['profitpercent_2020adj'], 10, labels=q10_labels)

In [ ]:
df['roi_quantile'].value_counts()

In [ ]:
qdict_roi = df['profitpercent_2020adj'].quantile(
    [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]).to_dict()

qdict_roi

In [ ]:
df['profit_quantile'] = pd.qcut(df['profit_2020adj'], 10, labels=q10_labels)
df['profit_quantile'].value_counts()

In [ ]:
qdict_profit = df['profit_2020adj'].quantile(
    [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]).to_dict()

qdict_profit

In [ ]:
df['budget_quantile'] = pd.qcut(df['budget_2020adj'], 10, labels=q10_labels)
df['budget_quantile'].value_counts()

In [ ]:
df['budget_top25'] = pd.qcut(df['budget_2020adj'], q=[0, .75, 1], 
                                labels=['Bottom 75%', 'Top 25%'])
df['budget_top25'].value_counts()

In [ ]:
df['budget_bottom25'] = pd.qcut(df['budget_2020adj'], q=[0, .25, 1], 
                                labels=['Bottom 25%', 'Top 75%'])
df['budget_bottom25'].value_counts()

In [ ]:
qdict_budget = df['budget_2020adj'].quantile(
    [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]).to_dict()

qdict_budget

In [ ]:
# Where do the profits for 50% of movies fall?
q_dict2 = df['profit_2020adj'].quantile(
    [.25, .75]).to_dict()

q_dict2

In [ ]:
df['profit_2020adj'].describe()

In [ ]:
df['budget_2020adj'].describe()

## How much profit can you expect to make per movie?

### 2020 not withstanding, people DO still go to the movies.

The graph below shows median profit (box office revenue less production budget) for the last 10 years. Amounts are in 2020 US dollars, after adjustment for inflation.

The median profit for films released in 2020 is actually negative. We calculated profit using only box office revenue, and movie theaters were closed for most of 2020 due to the COVID-19 pandemic.

The 2020 results have been left as strong evidence that the data we're using to perform this analysis is representative enough to show trends. While the median profit certainly hasn't increased steadily (2013, 2016, and 2018 had some dips),  profits are generally increasing over time. 

Before the COVID-19 pandemic, box office revenues were healthy, even as original titles produced by companies--such as Amazon, Netflix and Hulu--were released directly to subscribers, bypassing theaters entirely.  



In [ ]:
jcm_qualpalette1=sns.color_palette(palette=['springgreen','dodgerblue',
                                           'violet','gold','coral',
                                           'slateblue'])

jcm_qualpalette2=sns.color_palette(palette=['darkgoldenrod','olive','sienna'
                                           , 'slategrey', 'silver'])


ms_qualpalette=sns.color_palette(palette=['#00A1F1','#7CBB00','#FFBB00'
                                         , '#F65314','#747474'])

In [ ]:
with plt.style.context('seaborn-poster'):
#    sns.set_palette(jcm_qualpalette)
    fig, ax = plt.subplots()

    sns.barplot(x=df['release_year'], y=df['profit_2020adj']
                , estimator=np.median, ax=ax, ci=68, palette=ms_qualpalette)

    ax.set_title('Average Box Office Profit since 2010');
    ax.set_xlabel('Release Year');
    ax.set_ylabel('Median Profit');

    ax.annotate("Amazon Studios\n and Netflix\nrelease first\noriginal films[1][2]", 
                xy=(5, 4.5*1e7),  xytext=(5, 6*1e7),
                arrowprops=dict(color='black', lw=2, arrowstyle='-[', ),
                horizontalalignment='center', size=14)
    
    #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
    #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
    ax.yaxis.set_major_formatter(plt.FuncFormatter(currency))

### Many of the most profitable movies are outliers

Per the previous chart and below, median profit fell within a range of \\$20 MM to \\$35 MM 2020 US dollars for the past 10 years. Even the average was only around \\$100 MM.

But when we look at the distributions within each year, we see that there are many values well above that, in the many hundreds of millions, or billions of dollars. However, these are statistical outliers.

This means that while it's certainly possible to make hundreds of millions or billions of dollars at the box office on a film, that isn't a common outcome.

In [ ]:
# Median across across all years in millions
df['profit_2020adj'].quantile(0.5)/1000000

In [ ]:
# median profits per year
df_yr_medprofit = df.groupby('release_year')['profit_2020adj'].median().reset_index()
df_yr_medprofit['profit_2020adj'] = df_yr_medprofit['profit_2020adj'
                                                   ].map(lambda x : currency_num(x))

df_yr_medprofit.rename(columns={'profit_2020adj': 'Median Profit', 
                                'release_year': 'Year Released'})

In [ ]:
# overall mean profit in millions
df['profit_2020adj'].mean()/1000000

In [ ]:
# mean profit per year
df_yr_meanprofit = df.groupby('release_year')['profit_2020adj'].mean().reset_index()
df_yr_meanprofit['profit_2020adj'] = df_yr_meanprofit['profit_2020adj'
                                                   ].map(lambda x : currency_num(x))

df_yr_meanprofit.rename(columns={'profit_2020adj': 'Mean Profit', 
                                'release_year': 'Year Released'}, inplace=True)

In [ ]:
df.groupby('release_year')['profit_2020adj'].median().reset_index()['profit_2020adj']

In [ ]:
# Variance in median profits over the 10 years
np.std(df.groupby('release_year')['profit_2020adj'].median().reset_index(
     )['profit_2020adj'])

In [ ]:
# Other stats for median profits over the 10 years
df.groupby('release_year')['profit_2020adj'].median().reset_index(
     )['profit_2020adj'].describe()

In [ ]:
df['profit_2020adj'].describe()

In [ ]:
with plt.style.context('seaborn-poster'):
    fig, ax = plt.subplots()
    
    # Source: https://seaborn.pydata.org/generated/seaborn.boxplot.html
    sns.boxplot(x=df['release_year'], y=df['profit_2020adj']
                , ax=ax, palette=ms_qualpalette)

    ax.set_title('Worldwide Box Office Profit Distribution by Year');
    ax.set_xlabel('Release Year');
    ax.set_ylabel('Profit');

    #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
    #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
    ax.yaxis.set_major_formatter(plt.FuncFormatter(currency))
    
    #ax.set_yticks(np.arange(-500000000, 3000000000, 250000000));
    ax.set_yticks(np.arange(-250000000, 3000000000, 250000000))

In [ ]:
with plt.style.context('seaborn-poster'):
    fig, ax = plt.subplots()
    
    avg = df['profit_2020adj'].mean()
    
    # Source: https://seaborn.pydata.org/generated/seaborn.boxplot.html
    sns.boxplot(x=df['release_year'], y=df['profit_2020adj']
                , ax=ax, showfliers=False, palette=ms_qualpalette)

    ax.set_title('Worldwide Box Office Profit Distribution by Year (No Outliers)');
    ax.set_xlabel('Release Year');
    ax.set_ylabel('Profit');

    #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
    #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
    ax.yaxis.set_major_formatter(plt.FuncFormatter(currency))
    
    ax.set_yticks(np.arange(-150000000, 450000000, 50000000))


# Topic 2: Characteristics of Profitable Movies

Are the more profitable movies released in a certain month?

December has the highest median profit for movies released in that month. This isn't surprising, given many movies seem to be released in December to catch the holiday season in the US. 

It would be a good idea to plan for movies to be released in December to maximize profitability, and November, May, June, and July are also good choices.

Interestingly, September is the month with the most number of movies released, and it also has the lowest median profit. I would not recommend releasing movies in September, if you can avoid it.



In [ ]:
with plt.style.context('seaborn-poster'):
    g = sns.lmplot(x="budget_2020adj", y="profit_2020adj", 
                   height=8, 
                   data=df, scatter_kws = {'s': 25}
                  )

    for ax in g.axes.flat:

        ax.set_title('Production Budget versus Profit');
        ax.set_xlabel('Production Budget');
        ax.set_ylabel('Profit');

        #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
        #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        ax.xaxis.set_major_formatter(plt.FuncFormatter(currency))
        ax.yaxis.set_major_formatter(plt.FuncFormatter(currency))

In [ ]:
df['budget_2020adj'].describe()

In [ ]:
with plt.style.context('seaborn-poster'):
    g = sns.lmplot(x="budget_2020adj", y="profit_2020adj", 
                   # col_wrap=1, col='budget_top25', 
                   height=8,
                   data=df, scatter_kws = {'s': 25}
                  )

    for ax in g.axes.flat:

        ax.set_title('Production Budget versus Profit');
        ax.set_xlabel('Production Budget');
        ax.set_ylabel('Profit');

        #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
        #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        ax.xaxis.set_major_formatter(plt.FuncFormatter(currency))
        ax.yaxis.set_major_formatter(plt.FuncFormatter(currency))
        ax.vlines(df['budget_2020adj'].quantile(.75), colors='red', 
                  ymin=-2*1e8, ymax=1.8*1e9, linestyles='dashed')
        ax.annotate("75% of budgets are\nunder $62.73 M", color='red', 
                    xy=(df['budget_2020adj'].quantile(.75), 1.9*1e9),  
                    xytext=(df['budget_2020adj'].quantile(.75), 2.05*1e9),
                    arrowprops=dict(color='red', lw=2, arrowstyle='->', ),
                    horizontalalignment='center', size=16)

In [ ]:
df['budget_2020adj'].describe()

In [ ]:
with plt.style.context('seaborn-poster'):
    g = sns.lmplot(x="budget_2020adj", y="profitpercent_2020adj", 
                   #col='budget_bottom25', 
                   height=8,# col_wrap=1,
                   data=df, scatter_kws = {'s': 50}
                  )

    for ax in g.axes.flat:

        ax.set_title('Production Budget versus ROI');
        ax.set_xlabel('Production Budget');
        ax.set_ylabel('ROI %');

        #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
        #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        ax.xaxis.set_major_formatter(plt.FuncFormatter(currency))
        ax.vlines(df['budget_2020adj'].quantile(.25), colors='red', 
                  ymin=-100, ymax=40000, linestyles='dashed')
        ax.annotate("25% of budgets are\nunder $10.9 M", color='red', 
                    xy=(1e7, 40000),  
                    xytext=(1e8, 40000),
                    arrowprops=dict(color='red', lw=2, arrowstyle='->', ),
                    horizontalalignment='center', size=16)

In [ ]:
genres_topprofit = ['Adventure', 'Animation', 'Science Fiction', 
                    'Fantasy']
genres_toproi = ['Documentary', 'Horror', 'Animation', 'Comedy']

In [ ]:
with plt.style.context('seaborn-poster'):
    g = sns.lmplot(x="budget_2020adj", y="profit_2020adj", 
                   hue="genre", palette=ms_qualpalette,
                   height=10, #col_wrap=2,
                   data=df_genres[df_genres['genre'].isin(genres_topprofit)])

    for ax in g.axes.flat:

        ax.set_title('Production Budget versus Profit - Top 4 Genres');
        ax.set_xlabel('Production Budget');
        ax.set_ylabel('Profit');

        #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
        #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        ax.xaxis.set_major_formatter(plt.FuncFormatter(currency))
        ax.yaxis.set_major_formatter(plt.FuncFormatter(currency))

In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    # Source: https://stackoverflow.com/questions/12589481/multiple-aggregations-
    # of-the-same-column-using-pandas-groupby-agg
    profit_by_month = df.groupby('release_month_num').agg(
        Median=('profit_2020adj', 'median'), 
        Count=('profit_2020adj', 'count'))

    fig, ([ax2, ax]) = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))
    plt.tight_layout(pad=5)
    
    # Source: James Irving
    ax.plot(profit_by_month.index, profit_by_month.Count, marker='o', c='#00A1F1')
    ax2.plot(profit_by_month.index, profit_by_month.Median, marker='o', c='#F65314')
    
    # Specify we want to use seaborn to style the plot even though we're using
    # default matplotlib syntax
    # Source: https://www.codecademy.com/articles/seaborn-design-ii
    sns.set()

    # set and format labels
    ax.set_ylabel('Number of Movies');
    ax2.set_ylabel('Median Profit');
    ax.set_title('Movie Count per Release Month', 
                 fontdict={'fontsize':20})
    ax2.set_title('Median Profit per Release Month', 
                fontdict={'fontsize':20})

    ax.set_xticks(list(month_map.keys()));
    ax.set_xticklabels(list(month_map.values()), rotation=90);
    ax2.set_xticks(list(month_map.keys()));
    ax2.set_xticklabels(list(month_map.values()), rotation=90);
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(currency));

    

Let's look at genres, what are the most popular genres?

In [ ]:
sql2 = """SELECT mg.genre_id, g.genre, 
                 m.id, m.title
          FROM tmdb_movies m
          INNER JOIN tmdb_movie_genres mg ON mg.id = m.id
          INNER JOIN tmdb_genres g ON mg.genre_id = g.genre_id
         """

dfg = pd.DataFrame(cur.execute(sql2).fetchall(),columns=[x[0] for x in cur.description])

In [ ]:
dfg.info()

In [ ]:
df.info()

In [ ]:
# Merge the calculated values from our movies dataframe into the genres one
df_genres= dfg.merge(df[['id','profit_2020adj', 
                         'profitpercent_2020adj','budget_2020adj', 
                         'release_month_num']], on='id')

df_genres.head()

In [ ]:
# Create a pivot table showing aggregate profits, grouped by genre
df_gp = pd.pivot_table(data=df_genres, index='genre', 
                       aggfunc={'id': 'count',
                                'profit_2020adj': [np.sum, np.mean],
                                'profitpercent_2020adj': np.median} )

In [ ]:
# Movies with an Adventure genre win for mean profit per movie.
# SciFi comes next, with Animation, Fantasy, and Family next.
# Action is 6th.

g_meanprofit = df_gp['profit_2020adj', 'mean'].sort_values(ascending=False)[:10]
    
sns.set_context('talk')

fig, ax = plt.subplots(figsize=(5, 3));

sns.barplot(y=g_meanprofit.index, x=g_meanprofit, ax=ax, orient='h', 
            palette=ms_qualpalette);

ax.set_title('Top 10 Genres by Profit per Movie');
ax.set_xlabel('Mean Profit per Movie');
ax.set_ylabel('Genre');

#Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
#                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
ax.xaxis.set_major_formatter(plt.FuncFormatter(currency));
ax.tick_params(axis='x', labelsize=14, labelrotation=90, pad=0)

In [ ]:
# Documentaries have the highest ROI, followed by Horror and Animation.
# This kinda makes sense because documentaries will tend to have lower budgets 
# than big action or even animation films, so the ROI on a modest investment
# multiples more quickly.
# note that the genres that made the most profits hover around 125% to 144% ROI
# range. This is likely because they have higher budgets.

g_roi = df_gp['profitpercent_2020adj', 'median'].sort_values(ascending=False)[:10]

sns.set_context('talk')

fig, ax = plt.subplots(figsize=(5, 3));

sns.barplot(y=g_roi.index, x=g_roi, ax=ax, orient='h', 
            palette=ms_qualpalette);

ax.set_title('Top 10 Genres by ROI');
ax.set_xlabel('Median ROI per Movie');
ax.set_ylabel('Genre');

#Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
#                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p : '{:1.0f}%'.format(x)));
ax.tick_params(axis='x', labelsize=14, labelrotation=90, pad=0)

# Topic 3: Impact of Contributors on Profitability

## Get the data from SQL

In [ ]:
sql3 = """SELECT w.tconst, n.primary_name, n.birth_year, 'Writer' AS role
          FROM imdb_writers w
          LEFT JOIN imdb_names n ON w.nconst = n.nconst
"""


df_writers = pd.DataFrame(cur.execute(sql3).fetchall(),columns=[x[0] for x in cur.description])
df_writers.head()

In [ ]:
sql4 = """SELECT d.tconst, n.primary_name, n.birth_year, 'Director' AS role
          FROM imdb_directors d
          LEFT JOIN imdb_names n ON d.nconst = n.nconst
"""


df_dirs = pd.DataFrame(cur.execute(sql4).fetchall(),columns=[x[0] for x in cur.description])
df_dirs.head()

In [ ]:
sql5 = """SELECT p.tconst, n.primary_name, n.birth_year, p.category AS role
          FROM imdb_principals p
          LEFT JOIN imdb_names n ON n.nconst = p.nconst
          WHERE p.category IN ('actor', 'actress')
"""


df_prins = pd.DataFrame(cur.execute(sql5).fetchall(),columns=[x[0] for x in cur.description])
df_prins.head()

In [ ]:
sql6 = """SELECT r.tconst, r.averagerating, r.numvotes
          FROM imdb_ratings r
"""


df_imdbratings = pd.DataFrame(cur.execute(sql6).fetchall(),columns=[x[0] for x in cur.description])
df_imdbratings.head()

In [ ]:
display(len(df_writers), len(df_dirs), len(df_prins))

In [ ]:
df_roles = pd.concat([df_writers, df_dirs, df_prins])

len(df_roles)

In [ ]:
# Normalize roles to title case
df_roles['role'] = df_roles['role'].map(lambda x: x.title())

In [ ]:
df_roles['role'].unique()

In [ ]:
df_roles.rename(columns={'tconst': 'imdb_id'}, inplace=True)
df_imdbratings.rename(columns={'tconst': 'imdb_id'}, inplace=True)

In [ ]:
df.info()

In [ ]:
# Merge the imdb ratings and numvotes into tmdb movie working dataframe
df = df.merge(df_imdbratings[['imdb_id','averagerating', 'numvotes']], 
              how='left', on='imdb_id')

In [ ]:
# and rename to reflect the provenance
df.rename(columns={'averagerating': 'imdb_averagerating', 
                   'numvotes': 'imdb_numvotes'}, inplace=True)

In [ ]:
df.info()

In [ ]:
df_roles.info()

In [ ]:
# Merge the values from our movies dataframe into the roles one
df_credits = df_roles.merge(df[['imdb_id', 'id','profit_2020adj', 'profitpercent_2020adj',
                                'imdb_averagerating', 'imdb_numvotes', 'vote_average',
                                'vote_count']], 
                            on='imdb_id')

df_credits.info()

## Pivot data for analysis

In [ ]:
df_cred_pvt = pd.pivot_table(data=df_credits, index=['role', 'primary_name'], 
                       aggfunc={'id': 'count',
                                'profit_2020adj': [np.sum, np.mean],
                                'profitpercent_2020adj': np.mean,
                                'imdb_averagerating': np.mean,
                                'imdb_numvotes': np.min,
                                'vote_average': np.mean,
                                'vote_count': np.min
                               } )

In [ ]:
df_cred_pvt

In [ ]:
# Calculate 99th percentile cutoffs to find contributors to the truly top movies
profit_99cutoff = df['profit_2020adj'].quantile(.99)
print(profit_99cutoff)

roi_99cutoff = df['profitpercent_2020adj'].quantile(.99)
print(roi_99cutoff)

rate_99cutoff = df['imdb_averagerating'].quantile(.99)
print(rate_99cutoff)

## Formatting functions

In [ ]:
# function to apply a blue font color to cells where the value is over a threshold
def overcurrlimit_blue(val, limit):
    # Source: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
    
    val_list= str.split(val, ' ')
    
    # Convert rounded abbreviated amount back to full value
    if val_list[1] == "B":
        val_float = float(val_list[0][1:])*1e9
    elif val_list[1] == "M":
        val_float = float(val_list[0][1:])*1e6
        
    color='blue' if val_float > limit else 'black'
    return 'color: %s' % color

In [ ]:
# function to apply a blue font color to cells where the value is over a threshold
def overperlimit_blue(val, limit):
    # Source: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
        
    color='blue' if float(val[0:-1]) > limit else 'black'
    return 'color: %s' % color

In [ ]:
# function to apply a blue font color to cells where the value is over a threshold
def overratelimit_blue(val, limit):
    # Source: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
        
    color='blue' if val > limit else 'black'
    return 'color: %s' % color

## Functions for top ten

In [ ]:
# Top 10 directors based on average profit per movie from the past 10 years
def gettopten_bymeanprofit(df_pivot, roles):
    
    dfs=[]
    
    for role in roles:

        # Create subdf from pivot listing only the top 10 in specified role
        # by mean profit
        top_df = df_pivot.loc[(role)][[('profit_2020adj', 'mean'),
                                                ('id','count')]].sort_values(
            by=[('profit_2020adj', 'mean')], ascending=False)[:10]

        # remove the top level of the hierarchical index
        top_df.columns = top_df.columns.get_level_values(1)

        # Make primary_name a column instead of the index
        top_df.reset_index(inplace=True)

        # rename the columns for presentation
        top_df.rename(columns={'mean':'Mean Movie Profit', 'count': 'Movie Count',
                             'primary_name': role + ' Name'}, inplace=True)

        # redo the index to start at 1 for presentation
        top_df.index = (range(1, len(top_df)+1))

        # Apply currency format to mean profit column
        top_df['Mean Movie Profit'] = top_df['Mean Movie Profit'].apply(currency)
        
        dfs.append(top_df)

    return dfs


In [ ]:
# Top 10 directors based on mean ROI from the past 10 years
def gettopten_bymeanroi(df_pivot, roles):
    
    dfs=[]
    
    for role in roles:

        # Create subdf from pivot listing only the top 10 in specified role
        # by mean roi
        top_df = df_pivot.loc[(role)][[('profitpercent_2020adj', 'mean'),
            ('id','count')]].sort_values(by=[('profitpercent_2020adj', 'mean')
                                            ], ascending=False)[:10]

        # remove the top level of the hierarchical index
        top_df.columns = top_df.columns.get_level_values(1)

        # Make primary_name a column instead of the index
        top_df.reset_index(inplace=True)

        # rename the columns for presentation
        top_df.rename(columns={'mean':'Mean Movie ROI', 'count': 'Movie Count',
                             'primary_name': role + ' Name'}, inplace=True)

        # redo the index to start at 1 for presentation
        top_df.index = (range(1, len(top_df)+1))

        # Apply percent format to mean ROI column
        top_df['Mean Movie ROI'] = top_df['Mean Movie ROI'].apply(lambda x : f'{np.round(x,1)}%')

        dfs.append(top_df)

    return dfs


In [ ]:
# Top 10 directors based on average user rating on all movies from the past 10 years
def gettopten_bymeanrate(df_pivot, roles):

    dfs=[]
    
    for role in roles:
        # Create subdf from pivot listing only the top 10 in specified role
        # by mean rating
        top_df = df_pivot.loc[(role)][[('imdb_averagerating', 'mean'),('id','count'
              )]].sort_values(by=[('imdb_averagerating', 'mean')], ascending=False)[:10]

        # remove the top level of the hierarchical index
        top_df.columns = top_df.columns.get_level_values(1)

        # Make primary_name a column instead of the index
        top_df.reset_index(inplace=True)

        # rename the columns for presentation
        top_df.rename(columns={'mean':'Mean Movie Rating', 'count': 'Movie Count',
                             'primary_name':role + ' Name'}, inplace=True)

        # redo the index to start at 1 for presentation
        top_df.index = (range(1, len(top_df)+1))

        dfs.append(top_df)

    return dfs

## Functions for in top 1% with count greater than 1

In [ ]:
# Top 10 directors based on average profit per movie from the past 10 years
def gettoppercent_bymeanprofit(df_pivot, roles):
    
    dfs=[]
    
    for role in roles:

        # Create subdf from pivot listing only ones who worked on multiple
        # movies, where the mean profit for those movies is more than 99% of
        # all the movies we analyzed
        top_df = df_pivot.loc[(role)][[(
            'profit_2020adj', 'mean'),(
            'id','count')]]
        top_df = top_df.loc[(
            top_df[('profit_2020adj', 'mean')] > profit_99cutoff) & (
            top_df[('id','count')] > 1)]

        # remove the top level of the hierarchical index
        top_df.columns = top_df.columns.get_level_values(1)

        # Make primary_name a column instead of the index
        top_df.reset_index(inplace=True)

        # rename the columns for presentation
        top_df.rename(columns={'mean':'Mean Movie Profit', 'count': 'Movie Count',
                             'primary_name': role + ' Name'}, inplace=True)

        # redo the index to start at 1 for presentation
        top_df.index = (range(1, len(top_df)+1))

        # Apply currency format to mean profit column
        top_df['Mean Movie Profit'] = top_df['Mean Movie Profit'].apply(currency)
        
        dfs.append(top_df)

    return dfs


In [ ]:
# Top 10 directors based on mean ROI from the past 10 years
def gettoppercent_bymeanroi(df_pivot, roles):
    
    dfs=[]
    
    for role in roles:

        # Create subdf from pivot listing only ones who worked on multiple
        # movies, where the mean roi for those movies is more than 99% of
        # all the movies we analyzed
        top_df = df_pivot.loc[(role)][[(
            'profitpercent_2020adj', 'mean'),(
            'id','count')]]
        top_df = top_df.loc[(
            top_df[('profitpercent_2020adj', 'mean')] > roi_99cutoff) & (
            top_df[('id','count')] > 1)]

        # remove the top level of the hierarchical index
        top_df.columns = top_df.columns.get_level_values(1)

        # Make primary_name a column instead of the index
        top_df.reset_index(inplace=True)

        # rename the columns for presentation
        top_df.rename(columns={'mean':'Mean Movie ROI', 'count': 'Movie Count',
                             'primary_name': role + ' Name'}, inplace=True)

        # redo the index to start at 1 for presentation
        top_df.index = (range(1, len(top_df)+1))

        # Apply percent format to mean ROI column
        top_df['Mean Movie ROI'] = top_df['Mean Movie ROI'].apply(lambda x : f'{np.round(x,1)}%')

        dfs.append(top_df)

    return dfs


In [ ]:
# Top 10 directors based on average user rating on all movies from the past 10 years
def gettoppercent_bymeanrate(df_pivot, roles):

    dfs=[]
    
    for role in roles:
        # Create subdf from pivot listing only ones who worked on multiple
        # movies, where the mean rating for those movies is more than 99% of
        # all the movies we analyzed
        top_df = df_pivot.loc[(role)][[(
            'imdb_averagerating', 'mean'),(
            'id','count')]]
        top_df = top_df.loc[(
            top_df[('imdb_averagerating', 'mean')] > rate_99cutoff) & (
            top_df[('id','count')] > 1)]


        # remove the top level of the hierarchical index
        top_df.columns = top_df.columns.get_level_values(1)

        # Make primary_name a column instead of the index
        top_df.reset_index(inplace=True)

        # rename the columns for presentation
        top_df.rename(columns={'mean':'Mean Movie Rating', 'count': 'Movie Count',
                             'primary_name':role + ' Name'}, inplace=True)

        # redo the index to start at 1 for presentation
        top_df.index = (range(1, len(top_df)+1))

        dfs.append(top_df)

    return dfs

## Get tables

In [ ]:
# Get contributors who worked on multiple movies in the past ten years,
# and the average profit of their movies is higher than 99% of movies we analyzed
dfs_pres = gettoppercent_bymeanprofit(df_cred_pvt, [
    'Director', 'Writer', 'Actor', 'Actress'])


In [ ]:
# Top directors by mean movie profit
dfs_pres[0]

In [ ]:
# Top writers by mean movie profit
dfs_pres[1]

In [ ]:
# Top actors by mean movie profit
dfs_pres[2]

In [ ]:
# Top actresses by mean movie profit
dfs_pres[3]

In [ ]:
# Get contributors who worked on multiple movies in the past ten years,
# and the average profit of their movies is higher than 99% of movies we analyzed
dfs_pres2 = gettoppercent_bymeanroi(df_cred_pvt, [
    'Director', 'Writer', 'Actor', 'Actress'])

In [ ]:
# Top directors by mean movie roi
dfs_pres2[0]

In [ ]:
# Top writers by mean movie roi
dfs_pres2[1]

In [ ]:
# Top actors by mean movie roi
dfs_pres2[2]

In [ ]:
# Top actresses by mean movie roi
dfs_pres2[3]

In [ ]:
# Get contributors who worked on multiple movies in the past ten years,
# and the average profit of their movies is higher than 99% of movies we analyzed
dfs_pres3 = gettoppercent_bymeanrate(df_cred_pvt, [
    'Director', 'Writer', 'Actor', 'Actress'])

In [ ]:
# Top directors by mean movie rating
dfs_pres3[0]

In [ ]:
# Top writers by mean movie rating
dfs_pres3[1]

In [ ]:
# Top actors by mean movie rating
dfs_pres3[2]

In [ ]:
# Top actresses by mean movie rating
dfs_pres3[3]

# Extra stuff

In [ ]:
# I think this the most accurate representation of movies created by American
# production studios, using films associated with US production studios.
# After further analysis, also added movies that had English as original_language,
# which means this includes some english films produced by studios outside the US.
# Finally, limited to films with runtime over 60 mins, as was pulling in shorter
# films that may have a different audience.

sql2 = """SELECT DISTINCT m.id
                        , m.budget
                        , m.revenue
                        , m.release_date
                        , m.original_language
                        , m.imdb_id
                        , m.runtime
                        , m.vote_average
                        , m.vote_count
         FROM tmdb_movies m
         WHERE ((
                (m.id IN (SELECT DISTINCT pc.id 
                          FROM tmdb_movie_companies pc
                          INNER JOIN tmdb_prodco c ON c.prodco_id = pc.prodco_id
                          WHERE c.origin_country = 'US'))
            OR  (m.original_language = 'en')
               )
            AND (m.runtime > 60))
               
         """

df = pd.DataFrame(cur.execute(sql2).fetchall(),columns=[x[0] for x in cur.description])

In [ ]:
# Replace 0 with NULL values for budget, revenue, and runtime columns
# Do in SQL so don't have to redo each time unless added new data

sql4 = """UPDATE tmdb_movies
          SET budget = NULL WHERE budget = 0"""
sql5 = """UPDATE tmdb_movies
          SET revenue = NULL WHERE revenue = 0"""
sql6 = """UPDATE tmdb_movies
          SET runtime = NULL WHERE runtime = 0"""
sql7 = """UPDATE tmdb_movies
          SET release_date = NULL WHERE release_date = ''"""

#cur.execute(sql4)
#cur.execute(sql5)
#cur.execute(sql6)
#cur.execute(sql7)

conn.commit()

In [ ]:
# Take a look at just the non-nulls
df[df['release_date'].isna()==False]['release_date'].map(lambda x: len(x)).value_counts()

In [ ]:
# So these are mostly english-language films. What production companies 
# are we talking about for english-language films that aren't associated
# with a US production company?

# These are the production companies associated with films that NO US-based
# production company worked on, and where the original_language was English

sql3 = """SELECT DISTINCT pc.id, c.name AS prodco_name, c.origin_country
          FROM tmdb_movie_companies pc
          INNER JOIN tmdb_prodco c ON c.prodco_id = pc.prodco_id
          WHERE (
                 (pc.id NOT IN (SELECT DISTINCT pc.id 
                                FROM tmdb_movie_companies pc
                                INNER JOIN tmdb_prodco c ON c.prodco_id = pc.prodco_id
                                WHERE c.origin_country = 'US'))
             AND (pc.id IN (SELECT id FROM tmdb_movies 
                            WHERE original_language='en'))
                )
              """

df3 = pd.DataFrame(cur.execute(sql3).fetchall(),columns=[x[0] for x in cur.description])

In [ ]:
df3['prodco_name'].value_counts()

In [ ]:
df2['original_language'].value_counts()

In [ ]:
#Ultimately, I think I want the deduplicated union of both these sets.
# This represents all movies which were either 

In [ ]:
# release_decade
df['release_decade'] = df[df['release_year'] > 0]['release_year'].map(
    lambda x : int(str(x)[:3] + '0'))

df.loc[df['release_decade'].isna()==True, 'release_decade'] = 0

df['release_decade'] = df['release_decade'].astype(int)

In [ ]:
# Using only the data
# - release date in range 1960 to 2020
# - where we have a budget and it's greater than $10,000
# - where we have revenue

df_finance = df[(df['budget'] > 10000) & (df['revenue'].isna() == False)
   & (df['release_year'] >= 1960) & (df['release_year'] <= 2020)].copy()

In [ ]:
# Using only the data
# - release date in range 1960 to 2020
# - where we have a budget and it's greater than $10,000
# - where we have revenue

df_finance = df[(df['budget'] > 10000) & (df['revenue'].isna() == False)
   & (df['release_year'] >= 1960) & (df['release_year'] <= 2020)].copy()

In [ ]:
#df_recent = df_finance[(df_finance['release_year'] >= 2010) & (
#    df_finance['release_year'] < 2020)]

df_recent = df_finance[(df_finance['release_year'] >= 2010) & (
    df_finance['release_year'] <= 2020)]

In [ ]:
# Create % profit categories, my initial version
df.loc[df['profitpercent_2020adj'] <= -10, 'profit_category'] = 'Lost Money'
df.loc[(df['profitpercent_2020adj'] > -10) & (df['profitpercent_2020adj']
                                    <= 10), 'profit_category'] = '~ Broke Even'
df.loc[(df['profitpercent_2020adj'] > 10) & (df['profitpercent_2020adj']
                                    <= 50), 'profit_category'] = 'ROI 10-50%'
df.loc[(df['profitpercent_2020adj'] > 50) & (df['profitpercent_2020adj']
                                    <= 100), 'profit_category'] = 'ROI 50-100%'
df.loc[(df['profitpercent_2020adj'] > 100) & (df['profitpercent_2020adj']
                                    <= 500), 'profit_category'] = 'ROI 100-500%'
df.loc[(df['profitpercent_2020adj'] > 500) & (df['profitpercent_2020adj']
                                    <= 1000), 'profit_category'] = 'ROI 500-1000%'
df.loc[df['profitpercent_2020adj'] > 1000, 'profit_category'] = 'ROI Over 1000%'

In [ ]:
# Create % profit categories, updated version
# over 817% would be considered outliers
# everything below 0 is a loww
# amounts between 0 and outlier territory are split about equally into 4 bins
df.loc[df['profitpercent_2020adj'] < 0, 'profit_category'] = 'Loss'
df.loc[(df['profitpercent_2020adj'] >= 0) & (
    df['profitpercent_2020adj'] <= 200), 'profit_category'] = '0 - 200% ROI'
df.loc[(df['profitpercent_2020adj'] > 200) & (
    df['profitpercent_2020adj'] <= 400), 'profit_category'] = '200 - 400% ROI'
df.loc[(df['profitpercent_2020adj'] > 400) & (
    df['profitpercent_2020adj'] <= 600), 'profit_category'] = '400 - 600% ROI'
df.loc[(df['profitpercent_2020adj'] > 600) & (
    df['profitpercent_2020adj'] <= 800), 'profit_category'] = '600 - 800% ROI'
df.loc[df['profitpercent_2020adj'] > 800, 'profit_category'] = 'Over 800% ROI'


In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    fig, ax = plt.subplots()
    # source: https://stackoverflow.com/questions/57207245/
    #how-do-you-use-count-plot-and-line-plot-with-the-same-y-axis-in-seaborn
    ax2=ax.twinx()

    sns.countplot(x=df['release_month_num'], ax=ax)
    sns.lineplot(x=df['release_month_num'], y=df['profit_2020adj'], sort=True,
                 estimator=np.median, ax=ax2)
    
    #ax.plot(df.groupby('release_month')['profit_2020adj'].median());
    #ax.set_title('Count of Movies Released in each Calendar Month');
    #ax.set_xlabel('Release Month');
    #ax.set_ylabel('Number of Movies Released');

In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    fig, ([ax1, ax2]) = plt.subplots(ncols=1, nrows=2)

    sns.countplot(x=df['release_month_num'], ax=ax1)
    sns.lineplot(x=df['release_month_num'], y=df['profit_2020adj'], sort=True,
                 estimator=np.median, ax=ax2)
    
    #ax.plot(df.groupby('release_month')['profit_2020adj'].median());
    #ax.set_title('Count of Movies Released in each Calendar Month');
    #ax.set_xlabel('Release Month');
    #ax.set_ylabel('Number of Movies Released');

In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    fig, ax = plt.subplots()

    sns.countplot(x=df['release_month'], order=month_order)

    ax.set_title('Count of Movies Released in each Calendar Month');
    ax.set_xlabel('Release Month');
    ax.set_ylabel('Number of Movies Released');


In [ ]:
# Top 10 directors based on sum of the profits their movies made in the last 10 years
df_cred_pvt.loc[('Director')][[('profit_2020adj', 'sum'),('id','count'
      )]].sort_values(by=[('profit_2020adj', 'sum')], ascending=False)[:10]

In [ ]:
# Top 10 writers based on sum of the profits their movies made in the last 10 years
df_cred_pvt.loc[('Writer')]['profit_2020adj', 'sum'].sort_values(ascending=False)[:10]

In [ ]:
# Top 10 male actors based on sum of the profits their movies made in the last 10 years
df_cred_pvt.loc[('Actor')]['profit_2020adj', 'sum'].sort_values(ascending=False)[:10]

In [ ]:
# Top 10 actresses based on sum of the profits their movies made in the last 10 years
df_cred_pvt.loc[('Actress')]['profit_2020adj', 'sum'].sort_values(ascending=False)[:10]

In [ ]:
with plt.style.context('seaborn-poster'):
    sns.set()
    fig, ax = plt.subplots(figsize=(8, 8))

    sns.histplot(df['profit_2020adj'], ax=ax, bins=30, palette=ms_qualpalette)
    ax.set_title('Box Office Profit Distribution');
    ax.set_xlabel('Profit');

    #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
    #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
    ax.xaxis.set_major_formatter(plt.FuncFormatter(currency))

In [ ]:
profit_category_order = ['Loss', '0 - 200% ROI', '200 - 400% ROI', 
        '400 - 600% ROI', '600 - 800% ROI', 'Over 800% ROI']

with plt.style.context('seaborn-notebook'):
    plt.tight_layout();
    fig, ([ax1, ax2]) = plt.subplots(figsize=(8, 10), ncols=1, nrows=2);

    sns.scatterplot(x=df['budget_2020adj'], y=df['profit_2020adj']
                    , ax=ax1, hue=df['profit_category'],
                   hue_order=profit_category_order);
          
    sns.scatterplot(x=df['budget_2020adj'], y=df['profit_2020adj']
                    , ax=ax2, hue=df['profit_category'],
                   hue_order=profit_category_order,
                   palette='coolwarm');
    
    #ax1.set_title('Production Budget versus Box Office Profit (2020 US Dollars)');
    ax1.set_xlabel('Production Budget');
    ax1.set_ylabel('Box Office Profit');

    #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
    #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(currency));
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(currency));

In [ ]:
profit_category_order = q10_labels
quantile_palette=sns.color_palette(palette=['red', 'orange', 'yellow', 'greenyellow',
                                       'limegreen','forestgreen', 'darkolivegreen',
                                       'royalblue', 'indigo', 'fuchsia'])
# Let's slice out chunks of the percentiles and look at them separately

with plt.style.context('seaborn-notebook'):

    g = sns.lmplot(data=df, x='budget_2020adj', y='profit_2020adj', 
                   hue='profit_quantile', palette=quantile_palette,
                  height=8)


In [ ]:
g = sns.FacetGrid(df, row='profit_quantile')
g.map(sns.scatterplot, 'budget_2020adj', 'profit_2020adj')


In [ ]:
g = sns.FacetGrid(df, row='roi_quantile')
g.map(sns.scatterplot, 'budget_2020adj', 'profitpercent_2020adj')


In [ ]:
g = sns.FacetGrid(df, row='budget_quantile')
g.map(sns.scatterplot, 'budget_2020adj', 'profit_2020adj')


In [ ]:
profit_category_order = q10_labels
quantile_palette=sns.color_palette(palette=['red', 'orange', 'yellow', 'greenyellow',
                                       'limegreen','forestgreen', 'darkolivegreen',
                                       'royalblue', 'indigo', 'fuchsia'])
# Let's slice out chunks of the percentiles and look at them separately

with plt.style.context('seaborn-notebook'):

    g = sns.lmplot(data=df, x='budget_2020adj', y='profit_2020adj', 
                   hue='budget_quantile', palette=quantile_palette,
                  height=6)


In [ ]:
q10_labels

In [ ]:
size_dict={
 'Percentile 0-10': 10,
 'Percentile 10-20': 20,
 'Percentile 20-30': 30,
 'Percentile 30-40': 40,
 'Percentile 40-50': 50,
 'Percentile 50-60': 60,
 'Percentile 60-70': 70,
 'Percentile 70-80': 80,
 'Percentile 80-90': 90,
 'Percentile 90-100': 100}
quantile_palette=sns.color_palette(palette=['red', 'orange', 'yellow', 'greenyellow',
                                       'limegreen','forestgreen', 'darkolivegreen',
                                       'royalblue', 'indigo', 'fuchsia'])
# Let's slice out chunks of the percentiles and look at them separately

with plt.style.context('seaborn-poster'):

    sns.scatterplot(x=df['budget_2020adj'], y=df['profit_2020adj'], 
                    hue=df['budget_quantile'], size=df['budget_quantile'],
                    sizes=size_dict,
                    palette=quantile_palette )


In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    # Source: https://stackoverflow.com/questions/12589481/multiple-aggregations-
    # of-the-same-column-using-pandas-groupby-agg
    profit_by_month = df.groupby('release_month_num').agg(
        Median=('profit_2020adj', 'median'), 
        Count=('profit_2020adj', 'count'))

    profit_by_month

    # Specify we want to use seaborn to style the plot even though we're using
    # default matplotlib syntax
    # Source: https://www.codecademy.com/articles/seaborn-design-ii
    sns.set()

    fig, ax = plt.subplots(figsize=(8, 6))
    ax2=ax.twinx()
    # Source: James Irving
    ax.bar(profit_by_month.index, profit_by_month.Count)
    ax2.plot(profit_by_month.index,profit_by_month.Median,marker='o',c='r')


    # set and format labels
    ax.set_ylabel('Number of Movies Released');
    ax2.set_ylabel('Median Profit per Month');
    ax.set_title('Monthly')

    ax.set_xticks(list(month_map.keys()));
    ax.set_xticklabels(list(month_map.values()), rotation=90);
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(currency));

In [ ]:
# Movies with an Adventure genre have generated the most profit in the
# last 10 years, with Action, Comedy, and Science Fiction coming next.

g_sumprofit = df_gp['profit_2020adj', 'sum'].sort_values(ascending=False)[:10]

sns.set_context('talk')

fig, ax = plt.subplots(figsize=(5, 5));

sns.barplot(y=g_sumprofit.index, x=g_sumprofit, ax=ax, orient='h', 
            palette=ms_qualpalette);

ax.set_title('Top 10 Genres by Sum of Profit');
ax.set_xlabel('Sum of Profit (2010 through 2020)');
ax.set_ylabel('Genre');

#Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
#                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
ax.xaxis.set_major_formatter(plt.FuncFormatter(currency));
ax.tick_params(axis='x', labelsize=14, labelrotation=90, pad=0)

In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    # Source: https://stackoverflow.com/questions/12589481/multiple-aggregations-
    # of-the-same-column-using-pandas-groupby-agg
    profit_by_month = df.groupby('release_month_num').agg(
        Median=('profit_2020adj', 'median'), 
        Count=('profit_2020adj', 'count'))

    fig, ax = plt.subplots(figsize=(10, 4))
    ax2=ax.twinx()
    
    # Source: James Irving
    ax.bar(profit_by_month.index, profit_by_month.Count, 
           color=['#F65314'])#,'#7CBB00','#FFBB00', '#F65314','#747474'])
           #edgecolor='k', linewidth='1.5')
    ax2.plot(profit_by_month.index,profit_by_month.Median,marker='o',c='k')
    
    # Specify we want to use seaborn to style the plot even though we're using
    # default matplotlib syntax
    # Source: https://www.codecademy.com/articles/seaborn-design-ii
    sns.set()
    
    # Source: https://intellipaat.com/community/34074/how-to-get-rid-of-grid
    # -lines-when-plotting-with-seaborn-pandas-with-secondaryy
    ax2.grid(False)

    # set and format labels
    ax.set_ylabel('Number of Movies Released');
    ax2.set_ylabel('Median Profit');
    ax.set_title('Count and Median Profit of Movies per Release Month', 
                 fontdict={'fontsize':20})

    ax.set_xticks(list(month_map.keys()));
    ax.set_xticklabels(list(month_map.values()), rotation=90);
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(currency));
    ax2
    
    

In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    # Source: https://stackoverflow.com/questions/12589481/multiple-aggregations-
    # of-the-same-column-using-pandas-groupby-agg
    profit_by_month = df.groupby('release_month_num').agg(
        Median=('profit_2020adj', 'median'), 
        Count=('profit_2020adj', 'count'))

    fig, ax2 = plt.subplots(figsize=(10, 6))
    ax=ax2.twinx()
    
    # Source: James Irving
    ax.plot(profit_by_month.index, profit_by_month.Count, marker='o', c='#747474')
    ax2.plot(profit_by_month.index, profit_by_month.Median, marker='o', c='#F65314')
    
    # Specify we want to use seaborn to style the plot even though we're using
    # default matplotlib syntax
    # Source: https://www.codecademy.com/articles/seaborn-design-ii
    sns.set()
    
    # Source: https://intellipaat.com/community/34074/how-to-get-rid-of-grid
    # -lines-when-plotting-with-seaborn-pandas-with-secondaryy
    ax.grid(False)

    # set and format labels
    ax.set_ylabel('Number of Movies Released');
    ax2.set_ylabel('Median Profit');
    #ax.set_title('Count and Median Profit of Movies per Release Month', 
    #             fontdict={'fontsize':20})

    ax2.set_xticks(list(month_map.keys()));
    ax2.set_xticklabels(list(month_map.values()), rotation=90);
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(currency));
    ax.legend(['Number of\nMovies Released'], loc=(0, 1), fontsize=14)
    ax2.legend(['Median \nProfit'], loc=(.82, 1), fontsize=14)
    
    

In [ ]:
month_order = ['January','February','March','April','May','June',
             'July','August','September','October','November',
             'December']

with plt.style.context('seaborn-poster'):
    # Source: https://stackoverflow.com/questions/12589481/multiple-aggregations-
    # of-the-same-column-using-pandas-groupby-agg
    profit_by_month = df.groupby('release_month_num').agg(
        Median=('profit_2020adj', 'median'), 
        Count=('profit_2020adj', 'count'))

    fig, ([ax1, ax2]) = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))
    
    ax1.bar(profit_by_month.index, profit_by_month.Count, 
           color=['#F65314'])
    ax2.plot(profit_by_month.index,profit_by_month.Median,marker='o',c='k')
    
    # Specify we want to use seaborn to style the plot even though we're using
    # default matplotlib syntax
    # Source: https://www.codecademy.com/articles/seaborn-design-ii
    sns.set()

    # set and format labels
    ax1.set_ylabel('Number of Movies Released');
    ax2.set_ylabel('Median Profit');
    ax1.set_title('Count of Movies per Release Month', 
                 fontdict={'fontsize':14})
    ax2.set_title('Median Profit per Release Month', 
                 fontdict={'fontsize':14})

    ax1.set_xticks(list(month_map.keys()));
    ax1.set_xticklabels(list(month_map.values()), rotation=90
                       , fontdict={'fontsize':14});
    
    ax2.set_xticks(list(month_map.keys()));
    ax2.set_xticklabels(list(month_map.values()), rotation=90
                        , fontdict={'fontsize':14});
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(currency));
    
    

In [ ]:
df['budget_2020adj'].quantile(q=[.125, .875])

In [ ]:
df['profit_2020adj'].quantile(q=[.125, .875])

In [ ]:
profit_category_order = q10_labels
roi_palette=sns.color_palette(palette=['red', 'orange', 'yellow', 'greenyellow',
                                       'limegreen','forestgreen', 'darkolivegreen',
                                       'royalblue', 'indigo', 'fuchsia'])

with plt.style.context('seaborn-poster'):
    plt.tight_layout();
    fig, ax1 = plt.subplots();

    sns.scatterplot(x=df['budget_2020adj'], y=df['profit_2020adj']
                    , ax=ax1, hue=df['roi_quantile'], palette=roi_palette,
                   hue_order=profit_category_order);

    ax1.set_title('Production Budget versus Box Office Profit');
    ax1.set_xlabel('Production Budget');
    ax1.set_ylabel('Box Office Profit');
    ax1.legend(title='ROI Percentiles', fontsize=16, title_fontsize=16, 
              edgecolor='black');
    ax1.hlines(0, colors='grey', xmin=0, xmax=3.5*1e8, linestyles='dashed')
    ax1.annotate("Broke Even", color='grey', xy=(3.75*1e8, 0),  xytext=(3.9*1e8, 0),
                 arrowprops=dict(color='grey', lw=2, arrowstyle='->', ),
                 verticalalignment='center', size=16)

    #Source: https://stackoverflow.com/questions/53747298/how-to-format-seaborn-
    #                matplotlib-axis-tick-labels-from-number-to-thousands-or-mi
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(currency));
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(currency));

In [ ]:
qdict_budget

In [ ]:
budget_percentiles=[]
min_v = currency(qdict_budget[0.0])

for i, x in enumerate(qdict_budget.items()):
    
    if i > 0:
        # new min is previous max unless first item
        min_v = max_v if i > 1 else min_v
        # new max is value
        max_v = currency(x[1])
                 
        budget_percentiles.append({q10_labels[i-1]: {'min':min_v, 'max':max_v}})
                 
    elif i == 10:
        min_v = max_v
        max_v = currency(qdict_budget[1.0])

        budget_percentiles.append({q10_labels[i-1]: {'min':min_v, 'max':max_v}})
        
budget_percentiles

In [ ]:
qdict_budget

In [ ]:
#with plt.style.context('seaborn-notebook'):
    
fig, ([ax1, ax2, ax3, ax4, ax5],  
      [ax6, ax7, ax8, ax9, ax10]
     ) = plt.subplots(figsize=(20, 8),
                      ncols=5, nrows=2, );
plt.tight_layout(h_pad=2, w_pad=2);

plots=[ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10]

for i, plot in enumerate(plots):

    percentile_text = list(budget_percentiles[i].keys())[0]

    sns.scatterplot(x=df.loc[df['budget_quantile']==percentile_text,'budget_2020adj'], 
        y=df['profit_2020adj'], ax=plot)
    title=f"Budget {percentile_text}\n{budget_percentiles[i][percentile_text]['min']} to {budget_percentiles[i][percentile_text]['max']}"
    plot.set_title(title, fontdict={'fontsize':18});
    plot.set_xlabel('Budget', fontdict={'fontsize':18});
    plot.set_ylabel('Profit', fontdict={'fontsize':18});

    #Source: https://www.delftstack.com/howto/matplotlib/how-to-hide-axis-
    #text-ticks-and-or-tick-labels-in-matplotlib/#xaxis.set_visiblefalse
    #%2fyaxis.set_visiblefalse-to-hide-matplotlib-axis-including-axis-label
    plot.xaxis.set_ticks([])
    plot.yaxis.set_ticks([])

In [ ]:
df_genres.info()

In [ ]:
#with plt.style.context('seaborn-notebook'):
    
fig, ([ax1, ax2, ax3],  
      [ax4, ax5, ax6], 
      [ax7, ax8, ax9]
     ) = plt.subplots(figsize=(12, 10),
                      ncols=3, nrows=3, );
plt.tight_layout(h_pad=2, w_pad=1);

plots=[ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9]

for i, plot in enumerate(plots):
    
    percentile_text = list(budget_percentiles[i].keys())[0]

    sns.scatterplot(x=df.loc[df['budget_quantile']==percentile_text,'budget_2020adj'], 
        y=df['profit_2020adj'], ax=plot)
    title=f"Budget {percentile_text}\n{budget_percentiles[i][percentile_text]['min']} to {budget_percentiles[i][percentile_text]['max']}"
    plot.set_title(title, fontdict={'fontsize':14});
    plot.set_xlabel('Budget', fontdict={'fontsize':14});
    plot.set_ylabel('Profit', fontdict={'fontsize':14});

    #Source: https://www.delftstack.com/howto/matplotlib/how-to-hide-axis-
    #text-ticks-and-or-tick-labels-in-matplotlib/#xaxis.set_visiblefalse
    #%2fyaxis.set_visiblefalse-to-hide-matplotlib-axis-including-axis-label
    plot.xaxis.set_ticks([])
    plot.yaxis.set_ticks([])

## Top ten by role

In [ ]:
# Top 10 directors based on sum of profit from the past 10 years
df_cred_pvt.loc[('Director')][[('profit_2020adj', 'sum'),('id','count'
      )]].sort_values(by=[('profit_2020adj', 'sum')], ascending=False)[:10]

In [ ]:
# Create top 10 dataframes by mean profit for the specified roles
dfs_pres = gettopten_bymeanprofit(df_cred_pvt, 
                                 ['Director', 'Writer', 'Actor', 'Actress'])


In [ ]:
# top ten directors by mean profit
dfs_pres[0].style.applymap(lambda x: overcurrlimit_blue(x, profit_99cutoff)
                          , subset='Mean Movie Profit')

In [ ]:
# top ten writers by mean profit
dfs_pres[1].style.applymap(lambda x: overcurrlimit_blue(x, profit_99cutoff)
                          , subset='Mean Movie Profit')

In [ ]:
# top ten actors by mean profit
dfs_pres[2].style.applymap(lambda x: overcurrlimit_blue(x, profit_99cutoff)
                          , subset='Mean Movie Profit')

In [ ]:
# top ten actresses by mean profit
dfs_pres[3].style.applymap(lambda x: overcurrlimit_blue(x, profit_99cutoff)
                          , subset='Mean Movie Profit')

In [ ]:
# Create top 10 dataframes by mean roi for the specified roles
dfs_pres = gettopten_bymeanroi(df_cred_pvt, 
                                 ['Director', 'Writer', 'Actor', 'Actress'])

In [ ]:
# Top ten directors based on mean roi
dfs_pres[0].style.applymap(lambda x: overperlimit_blue(x, roi_99cutoff)
                          , subset='Mean Movie ROI')

In [ ]:
# Top ten writers based on mean roi
dfs_pres[1].style.applymap(lambda x: overperlimit_blue(x, roi_99cutoff)
                          , subset='Mean Movie ROI')

In [ ]:
# Top ten actors based on mean roi
dfs_pres[2].style.applymap(lambda x: overperlimit_blue(x, roi_99cutoff)
                          , subset='Mean Movie ROI')

In [ ]:
# Top ten actresses based on mean roi
dfs_pres[3].style.applymap(lambda x: overperlimit_blue(x, roi_99cutoff)
                          , subset='Mean Movie ROI')

In [ ]:
# Create top 10 dataframes by mean rating for the specified roles
dfs_pres = gettopten_bymeanrate(df_cred_pvt, 
                                 ['Director', 'Writer', 'Actor', 'Actress'])

In [ ]:
# Top ten directors based on mean rating
dfs_pres[0].style.applymap(lambda x: overratelimit_blue(x, rate_99cutoff)
                      , subset='Mean Movie Rating')

In [ ]:
# Top ten writers based on mean rating
dfs_pres[1].style.applymap(lambda x: overratelimit_blue(x, rate_99cutoff)
                      , subset='Mean Movie Rating')

In [ ]:
# Top ten actors based on mean rating
dfs_pres[2].style.applymap(lambda x: overratelimit_blue(x, rate_99cutoff)
                      , subset='Mean Movie Rating')

In [ ]:
# Top ten actresses based on mean rating
dfs_pres[3].style.applymap(lambda x: overratelimit_blue(x, rate_99cutoff)
                      , subset='Mean Movie Rating')

In [ ]:
250000000